# Imports

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd
import seaborn as sns
from matplotlib.legend_handler import HandlerBase
from matplotlib.text import Text
from matplotlib.legend import Legend
from scipy.stats import ranksums, ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.power import TTestIndPower
import statsmodels.stats.multicomp as mc

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import os
import json

In [19]:
with open('data/paths.json','r') as f:
    paths = json.load(f)

data_path = paths["data_path"]

In [20]:
batch_corrected = True
nan_remove = True
scaler_term = 'minmax'
scaler = MinMaxScaler() if scaler_term=='minmax' else StandardScaler()
data_col_idx = 19 if batch_corrected else 16

test_enc_name = '_BatchCor'+str(batch_corrected)+'_NanRem'+str(nan_remove)+'_'+str(scaler_term)

print(test_enc_name)

_BatchCorTrue_NanRemTrue_minmax


In [21]:
# with open(os.path.join('results','significant_lipids_pls2_treatment'+test_enc_name+'.txt'),'r') as f:
    # lipid_names = f.read().splitlines()
# print(lipid_names)
# print(len(lipid_names))

In [22]:
# len(lipid_names)

In [23]:
sns.set(style = 'whitegrid')

# Load data

In [24]:
data_tmp = pd.read_excel(os.path.join(data_path,'raw_data.xlsx'))
data_tmp.head()
lipid_names = np.unique(data_tmp['Individual Lipid Species'])

In [25]:
metadata = data_tmp.iloc[:,:11]
if batch_corrected:
    data = pd.concat([metadata, data_tmp.iloc[:,19]],axis=1)
else:
    data = pd.concat([metadata, data_tmp.iloc[:,16]],axis=1)

data_stats = data.reset_index().fillna(1e-3*np.min(np.abs(data.iloc[:,-1])))
data_stats.head()

,index,Sample Number,Sample Submission Date,Sample Name,Sex,Tissue weight (mg),Treatment,Tissue Type,PND,Litter,Individual Lipid Species,Lipid Class,BATCH CORRECTED\nNormalized Peak Area (Peak Area of Lipid Species / (Peak Area of Internal Standard * Tissue weight))
0,0,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(14:0)+H,SM,0.000842
1,1,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(16:0)+H,SM,0.006390
2,2,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(18:0)+H,SM,0.036989
3,3,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(18:1)+H,SM,0.046320
4,4,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(20:0)+H,SM,0.001167


In [26]:
data = data_stats.loc[data_stats['Individual Lipid Species'].isin(lipid_names)].copy(deep=True)
data.columns = ['Index','SampleNumber', 'SampleSubmissionDate', 'SampleName', 'Sex', 'TissueWeight', 'Treatment', 'TissueType', 'PND', 'Litter', 'IndividualLipidSpecies', 'LipidClass', 'PeakArea']
data.head()

,Index,SampleNumber,SampleSubmissionDate,SampleName,Sex,TissueWeight,Treatment,TissueType,PND,Litter,IndividualLipidSpecies,LipidClass,PeakArea
0,0,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(14:0)+H,SM,0.000842
1,1,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(16:0)+H,SM,0.006390
2,2,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(18:0)+H,SM,0.036989
3,3,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(18:1)+H,SM,0.046320
4,4,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(20:0)+H,SM,0.001167


In [27]:
data['log_area'] = np.log10(data.iloc[:,-1])
data['log_scaled_area'] = scaler.fit_transform(data[['log_area']])
data.head()

,Index,SampleNumber,SampleSubmissionDate,SampleName,Sex,TissueWeight,Treatment,TissueType,PND,Litter,IndividualLipidSpecies,LipidClass,PeakArea,log_area,log_scaled_area
0,0,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(14:0)+H,SM,0.000842,-3.074916,0.647776
1,1,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(16:0)+H,SM,0.006390,-2.194512,0.719944
2,2,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(18:0)+H,SM,0.036989,-1.431924,0.782454
3,3,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(18:1)+H,SM,0.046320,-1.334233,0.790462
4,4,1,09/2021,C20M1S,M,56,control,striatum,30,C20,SM(20:0)+H,SM,0.001167,-2.932747,0.659430


# Manipulate data

Add a column to identify the date

In [28]:
def classify_date(input_date):
    if input_date == '09/2021':
        return 1
    else:
        return 2

data['dateId'] = data['SampleSubmissionDate'].apply(lambda x: classify_date(x))

# Single lipids statistics

## Define lipids list

In [29]:
lipids_list = data['IndividualLipidSpecies'].unique()
print(lipids_list.shape[0])

1141


## Analysis control vs treatment

In [30]:
import itertools
factor = 'Treatment'
results_all = []
for l in lipids_list:
    data_lipid = data[data['IndividualLipidSpecies'] == l]
    if data_lipid[factor].unique().size > 1: # Some lipids are not found in both classes
        for c in itertools.combinations(data_lipid[factor].unique(),2):
            a = data_lipid[data_lipid[factor] == c[0]]['log_scaled_area'].values
            b = data_lipid[data_lipid[factor] == c[1]]['log_scaled_area'].values
            pvalue = ttest_ind(a,b)[1]
            fc = np.mean(a)/np.mean(b)
            cd = np.abs((np.mean(a) - np.mean(b)))
            # cd /= np.sqrt(
            vv = (
                ((a.size - 1)*np.var(a) + (b.size - 1)*np.var(b)) / (a.size + b.size - 2)
            )
            cd /= vv
            n_samples = a.size
            delta = np.abs(np.mean(a)-np.mean(b))
            sp = TTestIndPower().power(effect_size=cd, nobs1=n_samples, alpha=0.05)
            results_all.append([l, fc, pvalue, c[0], c[1], cd, n_samples, sp, delta, vv])

colnames = ['lipid', 'fc', 'p', factor+'1', factor+'2', 'effectSize', 'sampleSize', 'statisticalPower', 'delta', 'variance']
results_all = pd.DataFrame(data=results_all, columns= colnames)
results_all.head()

/var/folders/15/_htnf9m52ys2vzcf0_z47qnc0000gn/T/ipykernel_95019/3297434063.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  fc = np.mean(a)/np.mean(b)
/var/folders/15/_htnf9m52ys2vzcf0_z47qnc0000gn/T/ipykernel_95019/3297434063.py:11: RuntimeWarning: invalid value encountered in scalar divide
  fc = np.mean(a)/np.mean(b)
/var/folders/15/_htnf9m52ys2vzcf0_z47qnc0000gn/T/ipykernel_95019/3297434063.py:17: RuntimeWarning: invalid value encountered in scalar divide
  cd /= vv


,lipid,fc,p,Treatment1,Treatment2,effectSize,sampleSize,statisticalPower,delta,variance
0,SM(14:0)+H,1.009251,0.032918,control,deltamethrin,46.900205,36,1.0,0.005770,0.000123
1,SM(16:0)+H,1.005129,0.162395,control,deltamethrin,31.444357,36,1.0,0.003623,0.000115
2,SM(18:0)+H,1.004324,0.367727,control,deltamethrin,14.029382,36,1.0,0.003348,0.000239
3,SM(18:1)+H,1.004092,0.152439,control,deltamethrin,37.577478,36,1.0,0.003183,0.000085
4,SM(20:0)+H,1.005482,0.574527,control,deltamethrin,5.140556,36,1.0,0.003537,0.000688


### Statistical power with average effectSize

In [35]:
zcrit_alpha = 1.96 # Z-critical-value for alpha=0.05
zcrit_beta = 0.84 # Z-critical-value for effectsize=0.8
def custom_ss(delta, variance):
    return (zcrit_alpha + zcrit_beta)**2 * (variance/delta**2)

In [36]:
es_mean = np.mean(results_all['effectSize'][results_all['p']<0.05])
delta_mean = np.mean(results_all['delta'][results_all['p']<0.05])
var_mean = np.mean(results_all['variance'][results_all['p']<0.05])
n_mice = 24
statisticalPower = TTestIndPower().power(effect_size=es_mean, nobs1=n_mice, alpha=0.05)
print(statisticalPower)

1.0


### Given an effect size, and a statistical power, determine n_mice

In [37]:
es_mean = 0.8
alpha = 0.05
statisticalPower = 0.9
n_mice_ideal = TTestIndPower().solve_power(effect_size=es_mean, alpha=alpha, power=statisticalPower)
n_mice_custom = custom_ss(delta_mean, var_mean)
print(n_mice_ideal)
print(n_mice_custom)

33.825543836843956
34.63704580930791
